#### Trabalho Final  - Top. em Redes de Computadores

PPGCC UFPI - 2024.2

Professor: Glauber Dias Gonçalves

Alunos: Ericksulino e Samuel Ribeiro

In [93]:
import os
import numpy as np
import csv
import json
import pandas as pd

from math import ceil
from glob import glob

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

### Coleta, Preparação dos Dados e Engenharia de Features


In [94]:
clientes = glob("C:/Users/samue/Downloads/open-data/Train/dash/*")
clientes_servidores = {'ba': 0, 'rj': 1, 'ce': 0, 'df': 1, 'es': 2, 'pi': 3}
X = []
y = []


In [91]:
for cliente in clientes:
  servidores = glob(os.path.join(cliente, '*'))

  for servidor in servidores:
    requisicoes = servidores = glob(os.path.join(servidor, '*'))

    dash_values = []
    for file_path in requisicoes:
      with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[:-1]:
            data = json.loads(line)
            dash_values.append({
                'timestamp': pd.to_datetime(data['timestamp'], unit='s'),
                'rate': data['rate']
            })

    dash_serie = pd.DataFrame(dash_values)
    dash_serie.set_index("timestamp", inplace=True)
    dash_serie = dash_serie.sort_index()

    dash_serie_5min = dash_serie.resample('5min').agg({'rate': ['mean', 'std', 'median']}).dropna()

    grouped = []
    for i in range(0, dash_serie_5min.shape[0], 12):
      diff = dash_serie_5min.iloc[i:i + 12].index.to_series().diff().dropna()
      if all(diff == pd.Timedelta('5min')) and dash_serie_5min.iloc[i:i + 12].shape[0] == 12:
        grouped.append(dash_serie_5min.iloc[i:i + 12])

    for group in grouped:
        X.append([])
        y.append([])

        cliente_nome = os.path.basename(cliente)
        servidor_nome = os.path.basename(servidor)
        X[-1].append(clientes_servidores[cliente_nome]) #cliente
        X[-1].append(clientes_servidores[servidor_nome]) #servidor
        X[-1].append(group[('rate', 'mean')].iloc[0:10].mean())
        X[-1].append(group[('rate', 'median')].iloc[0:10].mean())
        X[-1].append(group[('rate', 'std')].iloc[0:10].std())

        y[-1].append(group[('rate', 'mean')].iloc[10])
        y[-1].append(group[('rate', 'std')].iloc[10])
        y[-1].append(group[('rate', 'mean')].iloc[11])
        y[-1].append(group[('rate', 'std')].iloc[11])

In [92]:
X

[[0, 0, 192410.67333333334, 205377.9, 9917.595613666139],
 [0, 0, 189865.7, 197298.1, 10487.224856389777],
 [0, 0, 204693.80666666664, 204329.8, 10338.322440751946],
 [0, 0, 182126.39333333334, 188490.3, 13152.016805725112],
 [0, 0, 155672.26, 169006.0, 10104.016239527367],
 [0, 0, 200653.52666666667, 198842.9, 22553.607359526326],
 [0, 0, 242654.4533333333, 229174.1, 14729.32620442692],
 [0, 0, 245817.61333333334, 238017.1, 27876.25155918136],
 [0, 0, 165458.3, 177764.1, 6950.060044697387],
 [0, 0, 187405.8, 203191.1, 9329.949394010868],
 [0, 0, 194271.22666666665, 212836.4, 7420.089079059192],
 [0, 0, 185183.74666666664, 198503.4, 11835.95727123066],
 [0, 0, 237288.12000000002, 229007.8, 17573.22967087732],
 [0, 0, 182763.17333333334, 191257.8, 17570.70560241163],
 [0, 0, 146535.21333333335, 160368.8, 8305.055340025621],
 [0, 0, 191227.5, 210394.0, 12655.965673351497],
 [0, 0, 187358.26, 201495.4, 8384.897225743325],
 [0, 0, 194983.88666666666, 207249.2, 9604.004151707933],
 [0, 0, 2

In [84]:
X_np = np.array(Xx)
y_np = np.array(yy)

### Treinamento e Validação dos Modelos

#### Linear Regressor (Padrão)

In [95]:
fold_length = ceil(X_np.shape[0] / 5)
folds = []

for i in range(0, X_np.shape[0], fold_length):
    test_indices = slice(i, i + fold_length)
    train_mask = np.ones(X_np.shape[0], dtype=bool)
    train_mask[test_indices] = False

    folds.append({
        'X_train': X_np[train_mask],
        'y_train': y_np[train_mask],
        'X_test': X_np[test_indices],
        'y_test': y_np[test_indices]
    })

model = LinearRegression()

def calculate_mape_for_output(folds, output_index):
    mape = []
    for fold in folds:
        model.fit(fold['X_train'], fold['y_train'][:, output_index])
        y_pred = model.predict(fold['X_test'])
        mape.append(mean_absolute_percentage_error(fold['y_test'][:, output_index], y_pred))
    return np.mean(mape)

for output_index in range(y_np.shape[1]):
    mape = calculate_mape_for_output(folds, output_index)
    print(f"MAPE para saída {output_index}: {mape:.5f}")

MAPE para saída 0: 0.10155
MAPE para saída 1: 0.08340
MAPE para saída 2: 0.37464
MAPE para saída 3: 0.20309


In [96]:
test_jsons = glob("C:/Users/samue/Downloads/open-data/Test/*")
submission = []
features = []

for test_path in test_jsons:
    with open(test_path, 'r') as file:
        data = json.load(file)
        
        test_name = test_path.split('\\')[-1].split('.')[0]
        submission.append([test_name])

        rates = [np.array(dash['rate']) for dash in data['dash']]
        rates_mean = np.mean([rate.mean() for rate in rates])
        rates_median = np.mean([np.median(rate) for rate in rates])
        rates_std = np.mean([rate.std() for rate in rates])

        client_value = clientes_servidores.get(data['cliente'], 0)
        server_value = clientes_servidores.get(data['servidor'], 0)
        features.append([client_value, server_value, rates_mean, rates_median, rates_std])

features_np = np.array(features)
rf_regressor = LinearRegression()
Xs = [X_np] * 4
Feats = [features_np] * 4

# Treina e realiza predições para cada saída
for i in range(4):
    rf_regressor.fit(Xs[i], y_np[:, i])
    y_pred = rf_regressor.predict(Feats[i])

    for j, y_pred_value in enumerate(y_pred):
        submission[j].append(y_pred_value)

output_path = "submission0.csv"
with open(output_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'mean_1', 'stdev_1', 'mean_2', 'stdev_2'])
    writer.writerows(submission)

print(f"Arquivo '{output_path}' foi criado com sucesso!")


Arquivo 'submission0.csv' foi criado com sucesso!


#### RandomForestRegressor com GridSearchCV

In [97]:
fold_length = ceil(X_np.shape[0] / 5)
folds = []

for i in range(0, X_np.shape[0], fold_length):
    test_indices = slice(i, i + fold_length)  
    train_mask = np.ones(X_np.shape[0], dtype=bool)
    train_mask[test_indices] = False

    folds.append({
        'X_train': X_np[train_mask],
        'y_train': y_np[train_mask],
        'X_test': X_np[test_indices],
        'y_test': y_np[test_indices]
    })

def calculate_mape_with_rf(folds, output_index):
    mape = []
    best_params = None

    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    for fold in folds:
        rf = RandomForestRegressor(random_state=42)
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)

        grid_search.fit(fold['X_train'], fold['y_train'][:, output_index])

        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        y_pred = best_model.predict(fold['X_test'])
        mape.append(mean_absolute_percentage_error(fold['y_test'][:, output_index], y_pred))

    return np.mean(mape), best_params

for output_index in range(y_np.shape[1]):
    mape, best_params = calculate_mape_with_rf(folds, output_index)
    print(f"MAPE para saída {output_index}: {mape:.5f}")
    print(f"Melhores parâmetros para saída {output_index}: {best_params}")


MAPE para saída 0: 0.10652
Melhores parâmetros para saída 0: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
MAPE para saída 1: 0.08806
Melhores parâmetros para saída 1: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
MAPE para saída 2: 0.38544
Melhores parâmetros para saída 2: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
MAPE para saída 3: 0.21120
Melhores parâmetros para saída 3: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


In [98]:
test_jsons = glob("C:/Users/samue/Downloads/open-data/Test/*")

submission = []
features = []

for test_path in test_jsons:
    with open(test_path, 'r') as file:
        data = json.load(file)

        test_name = test_path.split('\\')[-1].split('.')[0]
        submission.append([test_name])

        rates = [np.array(dash['rate']) for dash in data['dash']]
        rates_mean = np.mean([rate.mean() for rate in rates])
        rates_median = np.mean([np.median(rate) for rate in rates])

        client_value = clientes_servidores.get(data['cliente'], 0)
        server_value = clientes_servidores.get(data['servidor'], 0)
        features.append([client_value, server_value, rates_mean, rates_median])

features_np = np.array(features)

best_params = [
    {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200},
    {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200},
    {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200},
    {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100},
]

models = [
    RandomForestRegressor(**best_params[0], random_state=42),
    RandomForestRegressor(**best_params[1], random_state=42),
    RandomForestRegressor(**best_params[2], random_state=42),
    RandomForestRegressor(**best_params[3], random_state=42),
]

for i, model in enumerate(models):
    model.fit(X_np, y_np[:, i])  
    y_pred = model.predict(features_np)

    for j, y_pred_value in enumerate(y_pred):
        submission[j].append(y_pred_value)

output_path = "submission2.csv"
with open(output_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'mean_1', 'stdev_1', 'mean_2', 'stdev_2'])
    writer.writerows(submission)
print(f"Arquivo '{output_path}' foi criado com sucesso!")

ValueError: X has 4 features, but RandomForestRegressor is expecting 5 features as input.

#### ElasticNet e Ridge
    -   ElasticNet com GridSearch
    -   Pipeline do Ridge com GridSearch

In [100]:
fold_length = ceil(X_np.shape[0] / 5)
folds = []

for i in range(0, X_np.shape[0], fold_length):
    test_indices = slice(i, i + fold_length)  
    train_mask = np.ones(X_np.shape[0], dtype=bool)
    train_mask[test_indices] = False  

    folds.append({
        'X_train': X_np[train_mask],
        'y_train': y_np[train_mask],
        'X_test': X_np[test_indices],
        'y_test': y_np[test_indices]
    })

# Configuração do GridSearchCV
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],  
    'l1_ratio': [0.1, 0.5, 0.7, 0.9],  # Combinação de L1 e L2
    'max_iter': [100, 500, 1000],
    'tol': [1e-3, 1e-4, 1e-5],
}

# Ajuste dinâmico de k baseado no número de características
n_features = X_np.shape[1]  
k = min(n_features, 5)

pipeline = Pipeline([
    ('scaler', StandardScaler()),                           # Normalização
    ('feature_selection', SelectKBest(f_regression, k=k)),  # Seleção dinâmica de características
    ('ridge', Ridge())                                      # Modelo Ridge
])

# Função para calcular o MAPE com GridSearch
def calculate_mape_with_gridsearch(folds, output_index):
    mape = []
    best_params = None

    for fold in folds:
        #grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)
        grid_search = GridSearchCV(ElasticNet(), param_grid, cv=3, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)
        grid_search.fit(fold['X_train'], fold['y_train'][:, output_index])

        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        y_pred = best_model.predict(fold['X_test'])
        mape.append(mean_absolute_percentage_error(fold['y_test'][:, output_index], y_pred))

    return np.mean(mape), best_params

# Cálculo do MAPE para cada saída
for output_index in range(y_np.shape[1]):
    mape, best_params = calculate_mape_with_gridsearch(folds, output_index)
    print(f"MAPE para saída {output_index}: {mape:.5f}")
    print(f"Melhores parâmetros para saída {output_index}: {best_params}")


c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.617e+12, tolerance: 5.626e+09
  model = cd_fast.enet_coordinate_descent(
c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.465e+12, tolerance: 5.071e+09
  model = cd_fast.enet_coordinate_descent(


MAPE para saída 0: 0.10116
Melhores parâmetros para saída 0: {'alpha': 0.01, 'l1_ratio': 0.1, 'max_iter': 100, 'tol': 0.001}


c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.138e+11, tolerance: 6.600e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.269e+11, tolerance: 7.246e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.828e+11, toleranc

MAPE para saída 1: 0.08444
Melhores parâmetros para saída 1: {'alpha': 1, 'l1_ratio': 0.7, 'max_iter': 100, 'tol': 0.001}


c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.757e+12, tolerance: 6.291e+09
  model = cd_fast.enet_coordinate_descent(
c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.427e+12, tolerance: 5.205e+09
  model = cd_fast.enet_coordinate_descent(
c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.439e+12, toleranc

MAPE para saída 2: 0.37462
Melhores parâmetros para saída 2: {'alpha': 0.1, 'l1_ratio': 0.9, 'max_iter': 500, 'tol': 0.001}


c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.347e+11, tolerance: 7.002e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.430e+11, tolerance: 7.485e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.091e+11, toleranc

MAPE para saída 3: 0.20694
Melhores parâmetros para saída 3: {'alpha': 0.1, 'l1_ratio': 0.9, 'max_iter': 100, 'tol': 0.001}


c:\Users\samue\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.194e+11, tolerance: 7.222e+08
  model = cd_fast.enet_coordinate_descent(


In [ ]:
test_jsons = glob("C:/Users/samue/Downloads/open-data/Test/*")

submission = []
features = []

for test_path in test_jsons:
    with open(test_path, 'r') as file:
        data = json.load(file)

        test_name = test_path.split('\\')[-1].split('.')[0]
        submission.append([test_name])

        rates = [np.array(dash['rate']) for dash in data['dash']]
        rates_mean = np.mean([rate.mean() for rate in rates])
        rates_median = np.mean([np.median(rate) for rate in rates])

        client_value = clientes_servidores.get(data['cliente'], 0)
        server_value = clientes_servidores.get(data['servidor'], 0)
        features.append([client_value, server_value, rates_mean, rates_median])

features_np = np.array(features)

best_params = [
    {'alpha': 1},
    {'alpha': 100},
    {'alpha': 0.01},
    {'alpha': 10}
]

models = [
    Ridge(**best_params[0]),
    Ridge(**best_params[1]),
    Ridge(**best_params[2]),
    Ridge(**best_params[3])
]

for i, model in enumerate(models):
    model.fit(X_np, y_np[:, i]) 
    y_pred = model.predict(features_np) 

    for j, y_pred_value in enumerate(y_pred):
        submission[j].append(y_pred_value)

output_path = "submission3.csv"
with open(output_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'mean_1', 'stdev_1', 'mean_2', 'stdev_2'])
    writer.writerows(submission)
print(f"Arquivo '{output_path}' foi criado com sucesso!")


#### ElasticNet com Pipeline e GridSearch

In [ ]:
fold_length = ceil(X_np.shape[0] / 5)
folds = []

for i in range(0, X_np.shape[0], fold_length):
    test_indices = slice(i, i + fold_length) 
    train_mask = np.ones(X_np.shape[0], dtype=bool)
    train_mask[test_indices] = False

    folds.append({
        'X_train': X_np[train_mask],
        'y_train': y_np[train_mask],
        'X_test': X_np[test_indices],
        'y_test': y_np[test_indices]
    })

# Configuração do GridSearchCV
param_grid = {
    'elasticnet__alpha': [0.1, 1, 10, 100, 1000],  # Regularização aumentada
    'elasticnet__l1_ratio': [0.1, 0.5, 0.7, 0.9],  # Combinação de L1 e L2
    'elasticnet__max_iter': [1000, 2000, 5000],    # Aumentado o número de iterações
    'elasticnet__tol': [1e-3, 1e-4],               # Tolerância
}

n_features = X_np.shape[1] 
k = min(n_features, 5)

pipeline = Pipeline([
    ('scaler', StandardScaler()),              # Normalização
    ('feature_selection', SelectKBest(f_regression, k=k)),  # Seleção dinâmica de características
    ('elasticnet', ElasticNet())              # Modelo ElasticNet
])

# Função para calcular o MAPE com GridSearch
def calculate_mape_with_gridsearch(folds, output_index):
    mape = []
    best_params = None

    for fold in folds:
        grid_search = GridSearchCV(
            estimator=pipeline, 
            param_grid=param_grid, 
            cv=3, 
            scoring='neg_mean_absolute_percentage_error', 
            n_jobs=-1
        )
        grid_search.fit(fold['X_train'], fold['y_train'][:, output_index])

        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        y_pred = best_model.predict(fold['X_test'])
        mape.append(mean_absolute_percentage_error(fold['y_test'][:, output_index], y_pred))
    return np.mean(mape), best_params

for output_index in range(y_np.shape[1]):
    mape, best_params = calculate_mape_with_gridsearch(folds, output_index)
    print(f"MAPE para saída {output_index}: {mape:.5f}")
    print(f"Melhores parâmetros para saída {output_index}: {best_params}")


MAPE para saída 0: 0.10230
Melhores parâmetros para saída 0: {'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9, 'elasticnet__max_iter': 1000, 'elasticnet__tol': 0.001}
MAPE para saída 1: 0.08329
Melhores parâmetros para saída 1: {'elasticnet__alpha': 1, 'elasticnet__l1_ratio': 0.9, 'elasticnet__max_iter': 1000, 'elasticnet__tol': 0.001}
MAPE para saída 2: 0.37375
Melhores parâmetros para saída 2: {'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.9, 'elasticnet__max_iter': 1000, 'elasticnet__tol': 0.001}
MAPE para saída 3: 0.20287
Melhores parâmetros para saída 3: {'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.7, 'elasticnet__max_iter': 1000, 'elasticnet__tol': 0.0001}


In [ ]:

# Caminho para os arquivos JSON de teste
test_jsons = glob("C:/Users/samue/Downloads/open-data/Test/*")

# Inicializa listas para os dados
submission = []
features = []

# Processa os arquivos JSON de teste
for test_path in test_jsons:
    with open(test_path, 'r') as file:
        data = json.load(file)

        # Adiciona o nome do arquivo na submissão
        test_name = test_path.split('\\')[-1].split('.')[0]
        submission.append([test_name])

        # Extrai as taxas (rate) dos "dash" no JSON
        rates = [np.array(dash['rate']) for dash in data['dash']]
        rates_mean = np.mean([rate.mean() for rate in rates])
        rates_median = np.mean([np.median(rate) for rate in rates])

        # Adiciona as características baseadas no cliente e servidor
        client_value = clientes_servidores.get(data['cliente'], 0)
        server_value = clientes_servidores.get(data['servidor'], 0)
        features.append([client_value, server_value, rates_mean, rates_median])

# Converte as características para NumPy
features_np = np.array(features)

# Normalização dos dados
scaler = StandardScaler()
features_np = scaler.fit_transform(features_np)
X_np = scaler.fit_transform(X_np)

# Hiperparâmetros ajustados para cada saída (com max_iter aumentado)
best_params = [
    {'alpha': 0.1, 'l1_ratio': 0.9, 'max_iter': 10000, 'tol': 0.001},
    {'alpha': 1, 'l1_ratio': 0.5, 'max_iter': 10000, 'tol': 0.001},
    {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 10000, 'tol': 0.0001},
    {'alpha': 10, 'l1_ratio': 0.9, 'max_iter': 10000, 'tol': 0.0001},
]

# Inicializa os modelos ElasticNet com os melhores hiperparâmetros para cada saída
models = [
    ElasticNet(**best_params[0]),
    ElasticNet(**best_params[1]),
    ElasticNet(**best_params[2]),
    ElasticNet(**best_params[3])
]

# Treina e realiza predições para cada saída
for i, model in enumerate(models):
    model.fit(X_np, y_np[:, i])  # Treina o modelo com os dados de treino
    y_pred = model.predict(features_np)  # Realiza as predições com as características calculadas

    # Adiciona as predições ao arquivo de submissão
    for j, y_pred_value in enumerate(y_pred):
        submission[j].append(y_pred_value)

# Salva o arquivo CSV de saída com os títulos
output_path = "submission4.csv"
with open(output_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Adiciona os títulos
    writer.writerow(['id', 'mean_1', 'stdev_1', 'mean_2', 'stdev_2'])
    # Escreve os dados
    writer.writerows(submission)

print(f"Arquivo '{output_path}' foi criado com sucesso!")


Arquivo 'output4.csv' foi criado com sucesso!


#### VotingRegressor (ElasticNet e GradientBoostingRegressor)

In [79]:
test_jsons = glob("C:/Users/samue/Downloads/open-data/Test/*")

submission = []
features = []

for test_path in test_jsons:
    with open(test_path, 'r') as file:
        data = json.load(file)

        test_name = test_path.split('\\')[-1].split('.')[0]
        submission.append([test_name])

        rates = [np.array(dash['rate']) for dash in data['dash']]
        rates_mean = np.mean([rate.mean() for rate in rates])
        rates_median = np.mean([np.median(rate) for rate in rates])

        client_value = clientes_servidores.get(data['cliente'], 0)
        server_value = clientes_servidores.get(data['servidor'], 0)
        features.append([client_value, server_value, rates_mean, rates_median])

features_np = np.array(features)
scaler = StandardScaler()
features_np = scaler.fit_transform(features_np)
X_np = scaler.fit_transform(X_np)

elasticnet_param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],  # Regularização
    'l1_ratio': [0.1, 0.5, 0.7, 0.9],  # Combinação de L1 e L2
    'max_iter': [100, 500, 1000],
    'tol': [1e-3, 1e-4, 1e-5],
}

gboost_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

def find_best_models(X_train, y_train):
    best_models = []

    for i in range(y_train.shape[1]):
        # ElasticNet
        enet_grid = GridSearchCV(
            ElasticNet(),
            elasticnet_param_grid,
            cv=3,
            scoring='neg_mean_absolute_percentage_error',
            n_jobs=-1
        )
        enet_grid.fit(X_train, y_train[:, i])
        elasticnet_best = enet_grid.best_estimator_

        # Gradient Boosting
        gboost_grid = GridSearchCV(
            GradientBoostingRegressor(random_state=42),
            gboost_param_grid,
            cv=3,
            scoring='neg_mean_absolute_percentage_error',
            n_jobs=-1
        )
        gboost_grid.fit(X_train, y_train[:, i])
        gboost_best = gboost_grid.best_estimator_

        # Voting Regressor
        ensamble_model = VotingRegressor([('elasticnet', elasticnet_best), ('gboost', gboost_best)])
        ensamble_model.fit(X_train, y_train[:, i])

        best_models.append(ensamble_model)

    return best_models

best_models = find_best_models(X_np, y_np)
for i, model in enumerate(best_models):
    y_pred = model.predict(features_np)  
    for j, y_pred_value in enumerate(y_pred):
        submission[j].append(y_pred_value)

output_path = "submission_advanced.csv"
with open(output_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'mean_1', 'stdev_1', 'mean_2', 'stdev_2'])
    writer.writerows(submission)

print(f"Arquivo '{output_path}' foi criado com sucesso!")


Arquivo 'output_advanced.csv' foi criado com sucesso!


#FIM